In [1]:
import math
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns 
from chinese_calendar import is_holiday,is_workday
%matplotlib  inline
plt.style.use('seaborn-darkgrid')
sns.set(style = 'darkgrid')

import warnings
warnings.filterwarnings("ignore")
plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号
from sklearn.preprocessing import StandardScaler
import lightgbm as lgb  
from sklearn.preprocessing import scale
import lightgbm as lgb
import xgboost as xgb 
from catboost import CatBoostRegressor
import time
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder

from autogluon.tabular import TabularDataset, TabularPredictor

from sklearn import preprocessing


In [2]:
#减小内存占用
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [25]:
df = pd.read_excel("C:/Users/Enoch/Desktop/泰迪杯2023/程序/1_4_2/特征衍生/特征衍生_天.xlsx")
#df['日期1'] = pd.to_datetime(df['日期1'])#将字符串对象“2018/1/14”转为日期时间对象“2018-01-14”
df

,order_date,sales_region_code,item_code,first_cate_code,second_cate_code,sales_chan_name,item_price,ord_qty,which_year,day_of_year,weekday,day_of_month,is_promotion_day,is_holiday,season,day_of_season
0,2015-09-01,104,22069,307,403,1,1114.0,19,2015,244,1,1,0,0,3,63
1,2015-09-01,104,20028,301,405,1,1012.0,12,2015,244,1,1,0,0,3,63
2,2015-09-02,104,21183,307,403,0,428.0,109,2015,245,2,2,0,0,3,64
3,2015-09-02,104,20448,308,404,0,962.0,3,2015,245,2,2,0,0,3,64
4,2015-09-02,104,21565,307,403,1,1400.0,3,2015,245,2,2,0,0,3,64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
597688,2018-12-20,102,20994,302,408,1,843.0,59,2018,354,3,20,0,0,0,81
597689,2018-12-20,102,21875,302,408,1,762.0,502,2018,354,3,20,0,0,0,81
597690,2018-12-20,102,20215,302,408,1,2013.0,106,2018,354,3,20,0,0,0,81
597691,2018-12-20,102,20195,302,408,1,2120.0,187,2018,354,3,20,0,0,0,81


In [26]:
df = reduce_mem_usage(df)

Mem. usage decreased to 18.24 Mb (75.0% reduction)


In [27]:
category_cols = df.columns.values.tolist()
category_cols

['order_date',
 'sales_region_code',
 'item_code',
 'first_cate_code',
 'second_cate_code',
 'sales_chan_name',
 'item_price',
 'ord_qty',
 'which_year',
 'day_of_year',
 'weekday',
 'day_of_month',
 'is_promotion_day',
 'is_holiday',
 'season',
 'day_of_season']

In [28]:
category_cols
# 查看分类变量的取值水平
df[category_cols].nunique()

order_date            1172
sales_region_code        5
item_code             1758
first_cate_code          8
second_cate_code        12
sales_chan_name          2
item_price           14365
ord_qty               2404
which_year               4
day_of_year            366
weekday                  7
day_of_month            31
is_promotion_day         2
is_holiday               2
season                   4
day_of_season           92
dtype: int64

In [34]:
# 假设 top_corr 是一个 Pandas Series 对象
df_value = pd.DataFrame(df[category_cols].nunique(), columns=['value_level'])

# 将索引列 'variable' 转换为普通列
df_value = df_value.reset_index()

# 修改列名
df_value.columns = ['variable', 'value_level']
df_value.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
variable,order_date,sales_region_code,item_code,first_cate_code,second_cate_code,sales_chan_name,item_price,ord_qty,which_year,day_of_year,weekday,day_of_month,is_promotion_day,is_holiday,season,day_of_season
value_level,1172,5,1758,8,12,2,14365,2404,4,366,7,31,2,2,4,92


In [7]:
def Binary_Cross_Combination(colNames, features):#离散变量交叉验证
    """
    分类变量两两组合交叉衍生函数
    
    :param colNames: 参与交叉衍生的列名称
    :param features: 原始数据集
    :param OneHot: 是否进行独热编码
    
    :return:交叉衍生后的新特征和新列名称
    """
    
    # 创建空列表存储器
    colNames_new_l = []
    features_new_l = []
    
    # 提取需要进行交叉组合的特征
    features = features[colNames]
    
    # 逐个创造新特征名称、新特征
    for col_index, col_name in enumerate(colNames):
        for col_sub_index in range(col_index+1, len(colNames)):
            
            newNames = col_name + '&' + colNames[col_sub_index]
            colNames_new_l.append(newNames)
            
            newDF = pd.Series(features[col_name].astype('str')  
                              + '&'
                              + features[colNames[col_sub_index]].astype('str'), 
                              name=col_name)
            features_new_l.append(newDF)
    
    # 拼接新特征矩阵
    features_new = pd.concat(features_new_l, axis=1)
    features_new.columns = colNames_new_l
    colNames_new = colNames_new_l
    
           
    return features_new, colNames_new

In [8]:
# 只提取一年的数据

In [9]:

# 提取目标字段
colNames = [
 'sales_region_code',
 'first_cate_code',
 'second_cate_code',
 'sales_chan_name',
 'item_price',
 'which_year',
 'day_of_year',
 'day_of_month',
 'day_of_season']


In [10]:
df['order_date'] = pd.to_datetime(df['order_date'])#将字符串对象“2018/1/14”转为日期时间对象“2018-01-14”
start_date = '2017-12-01'
end_date = '2018-04-01'
df_one_year = df.loc[(df['order_date'] >= start_date) & (df['order_date'] < end_date)]
df_one_year
del df

In [11]:
features_new, colNames_new = Binary_Cross_Combination(colNames, df_one_year)

In [12]:
features_new

,sales_region_code&first_cate_code,sales_region_code&second_cate_code,sales_region_code&sales_chan_name,sales_region_code&item_price,sales_region_code&which_year,sales_region_code&day_of_year,sales_region_code&day_of_month,sales_region_code&day_of_season,first_cate_code&second_cate_code,first_cate_code&sales_chan_name,...,item_price&which_year,item_price&day_of_year,item_price&day_of_month,item_price&day_of_season,which_year&day_of_year,which_year&day_of_month,which_year&day_of_season,day_of_year&day_of_month,day_of_year&day_of_season,day_of_month&day_of_season
359616,102&308,102&404,102&1,102&2551.0,102&2017,102&335,102&1,102&62,308&404,308&1,...,2551.0&2017,2551.0&335,2551.0&1,2551.0&62,2017&335,2017&1,2017&62,335&1,335&62,1&62
359617,103&306,103&407,103&1,103&1195.0,103&2017,103&335,103&1,103&62,306&407,306&1,...,1195.0&2017,1195.0&335,1195.0&1,1195.0&62,2017&335,2017&1,2017&62,335&1,335&62,1&62
359618,102&308,102&404,102&1,102&2879.0,102&2017,102&335,102&1,102&62,308&404,308&1,...,2879.0&2017,2879.0&335,2879.0&1,2879.0&62,2017&335,2017&1,2017&62,335&1,335&62,1&62
359619,102&307,102&403,102&1,102&854.0,102&2017,102&335,102&1,102&62,307&403,307&1,...,854.0&2017,854.0&335,854.0&1,854.0&62,2017&335,2017&1,2017&62,335&1,335&62,1&62
359620,102&308,102&404,102&1,102&1456.0,102&2017,102&335,102&1,102&62,308&404,308&1,...,1456.0&2017,1456.0&335,1456.0&1,1456.0&62,2017&335,2017&1,2017&62,335&1,335&62,1&62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
426604,102&306,102&407,102&1,102&821.0,102&2018,102&90,102&31,102&90,306&407,306&1,...,821.0&2018,821.0&90,821.0&31,821.0&90,2018&90,2018&31,2018&90,90&31,90&90,31&90
426605,102&306,102&407,102&1,102&453.0,102&2018,102&90,102&31,102&90,306&407,306&1,...,453.0&2018,453.0&90,453.0&31,453.0&90,2018&90,2018&31,2018&90,90&31,90&90,31&90
426606,102&302,102&408,102&1,102&5310.0,102&2018,102&90,102&31,102&90,302&408,302&1,...,5310.0&2018,5310.0&90,5310.0&31,5310.0&90,2018&90,2018&31,2018&90,90&31,90&90,31&90
426607,102&303,102&401,102&1,102&2249.0,102&2018,102&90,102&31,102&90,303&401,303&1,...,2249.0&2018,2249.0&90,2249.0&31,2249.0&90,2018&90,2018&31,2018&90,90&31,90&90,31&90


In [13]:
colNames_new

['sales_region_code&first_cate_code',
 'sales_region_code&second_cate_code',
 'sales_region_code&sales_chan_name',
 'sales_region_code&item_price',
 'sales_region_code&which_year',
 'sales_region_code&day_of_year',
 'sales_region_code&day_of_month',
 'sales_region_code&day_of_season',
 'first_cate_code&second_cate_code',
 'first_cate_code&sales_chan_name',
 'first_cate_code&item_price',
 'first_cate_code&which_year',
 'first_cate_code&day_of_year',
 'first_cate_code&day_of_month',
 'first_cate_code&day_of_season',
 'second_cate_code&sales_chan_name',
 'second_cate_code&item_price',
 'second_cate_code&which_year',
 'second_cate_code&day_of_year',
 'second_cate_code&day_of_month',
 'second_cate_code&day_of_season',
 'sales_chan_name&item_price',
 'sales_chan_name&which_year',
 'sales_chan_name&day_of_year',
 'sales_chan_name&day_of_month',
 'sales_chan_name&day_of_season',
 'item_price&which_year',
 'item_price&day_of_year',
 'item_price&day_of_month',
 'item_price&day_of_season',
 'whic

In [14]:
dummies_try = pd.get_dummies(features_new,prefix=colNames_new)
dummies_try

,sales_region_code&first_cate_code_101&301,sales_region_code&first_cate_code_101&302,sales_region_code&first_cate_code_101&303,sales_region_code&first_cate_code_101&305,sales_region_code&first_cate_code_101&306,sales_region_code&first_cate_code_101&307,sales_region_code&first_cate_code_101&308,sales_region_code&first_cate_code_102&301,sales_region_code&first_cate_code_102&302,sales_region_code&first_cate_code_102&303,...,day_of_month&day_of_season_7&68,day_of_month&day_of_season_7&7,day_of_month&day_of_season_8&39,day_of_month&day_of_season_8&67,day_of_month&day_of_season_8&69,day_of_month&day_of_season_8&8,day_of_month&day_of_season_9&40,day_of_month&day_of_season_9&68,day_of_month&day_of_season_9&70,day_of_month&day_of_season_9&9
359616,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
359617,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
359618,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
359619,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
359620,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
426604,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
426605,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
426606,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
426607,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [15]:
# 只提取一年

In [16]:
# 这将两个数据帧对象（df和dummies）按列进行拼接（axis=1），并将结果存储在df2新数据帧中。
df1 = pd.concat([df_one_year, dummies_try], axis=1)
# features_new = features_new+df["ord_qty"]
df1
del df_one_year
del dummies_try

In [30]:
#皮尔逊相关系数,特征选择
# 提取特征名称
# features = df1.columns.tolist()
# features = features[:2000]
# features.remove("order_date")
# features.remove("ord_qty")
# features.remove("item_code")
# featureSelect = features[:]
# featureSelect
"""
代码解释：

这段代码的作用是计算数据帧中特征与“总有功功率(kw)”之间的相关系数，并选择与之最相关的前300个特征进行建模。其中，“featureSelect”是一个包含所有特征的列表，而“df3”是数据帧对象。

首先，在for循环中，我们遍历了featureSelect列表中的每一个特征。对于每一个特征，我们使用“fillna(0)”方法将所有缺失值替换为0，然后使用“corr()”方法计算该特征与“总有功功率(kw)”之间的相关系数，并将其取绝对值。我们将这些相关系数添加到名为“corr”的空列表中。

接下来，我们使用pandas库中的Series对象创建一个名为“se”的序列，它包含了所有特征与“总有功功率(kw)”之间的相关系数。我们将这些相关系数按降序排序，并选取前300个特征。最后，我们将“日期1”和这300个特征组成一个名为“feature_select”的新列表。
"""
# 计算相关系数
# corr = []
# for fea in featureSelect:
#     corr.append(abs(df1[[fea, 'ord_qty']].fillna(0).corr().values[0][1]))

# # 取top300的特征进行建模，具体数量可选
# se = pd.Series(corr, index=featureSelect).sort_values(ascending=False)
# feature_select =['order_date'] +['item_code'] + se[:300].index.tolist()

# # 结果
# df1 = df1[feature_select + ['ord_qty']]
# df1
corr = []
corr = df1.corrwith(df1["ord_qty"], method="pearson")
top_corr = corr.abs().sort_values(ascending=False)[1:301]
top_corr

item_price                                 0.162409
first_cate_code                            0.146638
first_cate_code&sales_chan_name_303&1      0.115871
sales_region_code&sales_chan_name_105&1    0.112681
first_cate_code&sales_chan_name_307&1      0.108290
                                             ...   
sales_region_code&item_price_103&351.0     0.039523
item_price&day_of_season_369.0&31          0.039441
item_price&day_of_month_369.0&31           0.039441
item_price&day_of_year_369.0&31            0.039441
first_cate_code&item_price_307&365.0       0.039320
Length: 300, dtype: float64

In [31]:
# 假设 top_corr 是一个 Pandas Series 对象
df_top_corr = pd.DataFrame(top_corr, columns=['corr_abs'])

# 将索引列 'variable' 转换为普通列
df_top_corr = df_top_corr.reset_index()

# 修改列名
df_top_corr.columns = ['variable', 'corr_abs']
df_top_corr

,variable,corr_abs
0,item_price,0.162409
1,first_cate_code,0.146638
2,first_cate_code&sales_chan_name_303&1,0.115871
3,sales_region_code&sales_chan_name_105&1,0.112681
4,first_cate_code&sales_chan_name_307&1,0.108290
...,...,...
295,sales_region_code&item_price_103&351.0,0.039523
296,item_price&day_of_season_369.0&31,0.039441
297,item_price&day_of_month_369.0&31,0.039441
298,item_price&day_of_year_369.0&31,0.039441


In [32]:
corr_2 = []
corr_2 = df1.corrwith(df1["ord_qty"], method="spearman")
top_corr_2 = corr_2.abs().sort_values(ascending=False)[1:101]
top_corr_2

item_price                                 0.319414
sales_region_code&sales_chan_name_105&1    0.273813
first_cate_code                            0.188312
first_cate_code&sales_chan_name_303&1      0.169034
sales_region_code&sales_chan_name_103&1    0.133861
                                             ...   
first_cate_code&which_year_308&2017        0.068158
second_cate_code&which_year_404&2017       0.068158
second_cate_code&which_year_406&2017       0.068018
first_cate_code&which_year_305&2017        0.067457
second_cate_code&which_year_412&2017       0.067457
Length: 100, dtype: float64

In [33]:
# 假设 top_corr 是一个 Pandas Series 对象
df_top_corr_2 = pd.DataFrame(top_corr_2, columns=['corr_abs'])

# 将索引列 'variable' 转换为普通列
df_top_corr_2 = df_top_corr_2.reset_index()

# 修改列名
df_top_corr_2.columns = ['variable', 'corr_abs']
df_top_corr_2

,variable,corr_abs
0,item_price,0.319414
1,sales_region_code&sales_chan_name_105&1,0.273813
2,first_cate_code,0.188312
3,first_cate_code&sales_chan_name_303&1,0.169034
4,sales_region_code&sales_chan_name_103&1,0.133861
...,...,...
95,first_cate_code&which_year_308&2017,0.068158
96,second_cate_code&which_year_404&2017,0.068158
97,second_cate_code&which_year_406&2017,0.068018
98,first_cate_code&which_year_305&2017,0.067457


In [21]:
# top_corr
# top_corr_1 = top_corr[:200]
# top_corr_1
# feature_select =['order_date'] +['item_code'] + top_corr_1.index.tolist()+ ['ord_qty']

# # 结果
# df2 = df1.loc[:,feature_select]
# df2

In [22]:
# df1.to_csv("train_jiaocha.csv", index=False)

In [23]:
# col = df1.columns.values.tolist()
# col